In [27]:
import torch
import torch.nn as nn
import os

from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
import numpy as np 
import pandas as pd
import datetime
import pickle
import time

In [28]:
options = ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument("--user-data-dir=hkjc")
driver = Chrome(options=options)
driver.get("https://racing.hkjc.com/racing/information/English/Racing/RaceCard.aspx")
WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.ID, 'hplnkColSelect').is_displayed())

# driver.find_element(By.ID, 'hplnkColSelect').click()

# driver.find_element(By.XPATH, '//*[@id="ColSelectBody"]/form/table/tbody/tr[2]/td[3]/input').click()
# driver.find_element(By.XPATH, '//*[@id="ColSelectBody"]/form/table/tbody/tr[2]/td[4]/input').click()
# driver.find_element(By.XPATH, '//*[@id="ColSelectBody"]/form/table/tbody/tr[4]/td[5]/input').click()
# driver.find_element(By.XPATH, '//*[@id="ColSelectBody"]/form/table/tbody/tr[6]/td[2]/input').click()
# driver.find_element(By.XPATH, '//*[@id="ColSelectBody"]/form/table/tbody/tr[10]/td[3]/input').click()

# driver.find_element(By.XPATH, '//*[@id="ColSelectBody"]/form/table/thead/tr/td/table/tbody/tr/td[2]/a').click()
# driver.find_element(By.ID, 'hplnkColSelect').click()


True

In [29]:
def append_to_dict (dict, key, var):
    if key in dict.keys():
        dict[key].append(var)
    else:
        dict[key] = [var]
    return dict

def equalise_dict(dict):
    max_len = 0
    for key in dict:
        if len(dict[key]) > max_len:
            max_len = len(dict[key])
    for key in dict:
        if len(dict[key]) < max_len:
            for _ in range(max_len - len(dict[key])):
                dict[key].append(list(dict[key])[0])
    return dict

def lower_case (df):
    for c in df.columns:
        df[str(c).lower()] = df[c].apply(lambda x: str(x).lower())
        if str(c).lower() != c:
            df.drop(c, axis=1, inplace=True)
    return df

In [30]:
def scrape_jockey (driver, jockey_name):

    jockey_csv_path = os.path.join("data","1_scrape",'jockeys',f"{jockey_name}.csv")

    jockey_dict = {}

    for row in driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[2]/table/tbody/tr'):
        if 'age' in row.find_element(By.TAG_NAME, 'td').text.lower():
            var_value = row.find_element(By.TAG_NAME, 'td').text.lower().replace('age:','').strip()
            if 'background' not in var_value:
                jockey_dict = append_to_dict(jockey_dict, 'jockey_age', var_value)
        if'career wins' in row.find_element(By.TAG_NAME, 'td').text.lower():
            jockey_dict = append_to_dict(jockey_dict, 'career_wins', row.find_element(By.TAG_NAME, 'td').text.lower().strip())

    jockey_dict = append_to_dict(jockey_dict, 'nationality', driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]/td[2]').text.lower().replace(':','').strip())

    # season = driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/thead/tr/td').text.lower().split(' ')[0]
    for row in driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr'):
        num_cols = len(row.find_elements(By.TAG_NAME, 'td'))
        for i in range(int(num_cols/2)):
            # var_name = '_'.join([str(season), row.find_elements(By.TAG_NAME, 'td')[i].text.lower()])
            var_name = row.find_elements(By.TAG_NAME, 'td')[2 * i].text.lower()
            if 'nationality' in var_name:
                continue
            if var_name == '':
                continue
            var_value = row.find_elements(By.TAG_NAME, 'td')[2 * i + 1].text.lower().replace(':','').strip()
            jockey_dict = append_to_dict(jockey_dict, var_name, var_value)

    # if driver.find_elements(By.XPATH, '//*[@id="seasonTextId"]'):
    #     driver.find_element(By.XPATH, '//*[@id="seasonTextId"]').click()
    #     WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]/td[2]').is_displayed())
    #     season = driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/thead/tr/td').text.lower().split(' ')[0]
    #     for row in driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr'):
    #         num_cols = len(row.find_elements(By.TAG_NAME, 'td'))
    #         for i in range(num_cols/2):
    #             var_name = '_'.join([str(season), row.find_elements(By.TAG_NAME, 'td')[i].text.lower()])
    #             if 'nationality' in var_name:
    #                 continue
    #             if var_name == '':
    #                 continue
    #             var_value = row.find_elements(By.TAG_NAME, 'td')[i].text.lower().replace(':','').strip()
    #             jockey_dict = append_to_dict(jockey_dict, var_name, var_value)

    # click on performance
    driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[1]/div/ul[1]/li[2]/a').click()
    WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]').is_displayed())
    # click on previous season
    # if driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[2]/table/tbody/tr[8]/td/a'):
    #     driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[2]/table/tbody/tr[8]/td/a').click()
    #     WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]').is_displayed())
    for row in driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr'):
        cells = row.find_elements(By.TAG_NAME, 'td')
        if len(cells) == 7:
            var_name_1 = cells[0].text.lower()
            var_name_1 += '_' + cells[1].text.lower()
        elif len(cells) == 6:
            var_name_1 = cells[0].text.lower()
        for jockey_perf_idx in range(4):
            var_name = var_name_1 + '_' + cells[-5].text.lower() + '_' + driver.find_element(By.XPATH, f'//*[@id="innerContent"]/div[2]/div[3]/table/thead/tr/td[{jockey_perf_idx + 4}]').text.lower()
            var_value = cells[-jockey_perf_idx - 1].text
            jockey_dict = append_to_dict(jockey_dict, var_name, var_value)

    jockey_dict = equalise_dict(jockey_dict)
    jockey_df = pd.DataFrame(jockey_dict)
    jockey_df = lower_case(jockey_df)
    jockey_df.to_csv(jockey_csv_path)
    print(jockey_csv_path, 'saved')

In [31]:
try:
    original_window = driver.current_window_handle
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            driver.close()
    driver.switch_to.window(original_window)
except:
    pass

driver.get("https://racing.hkjc.com/racing/information/English/Racing/RaceCard.aspx")
WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.ID, 'hplnkColSelect').is_displayed())

race_buttons_indices = []

for cell_idx, cell in enumerate(driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]/td')):
    if len(cell.find_elements(By.TAG_NAME, 'img')) > 0:
        race_buttons_indices.append(cell_idx + 1)

for race_button_idx in race_buttons_indices:

    time.sleep(0.5)
    driver.find_element(By.XPATH, f'//*[@id="innerContent"]/div[2]/div[3]/table/tbody/tr[1]/td[{race_button_idx}]').click()
    WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="racecardlist"]/tbody/tr/td/table').is_displayed())

    race_dict = {'total_stakes':[],'horse_weight':[],'horse_handicap':[],'horse_odds':[],'horse_rating':[],'horse_import_type':[],'horse_sex':[],'horse_colour':[],
                'horse_age':[],'horse_origin':[],'horse_gear':[],'days_since_import':[],
                'draw':[],
                'jockey_age':[],'jockey_nationality':[],'jockey_wins':[],'jockey_rides':[],'jockey_stakes':[],'jockey_same_race_wins':[],
                'race_location':[],'race_class':[],'race_going':[],'race_distance':[],'race_surface':[],'race_course':[]}

    race_details = driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[4]/div[2]').text.splitlines()
    race_no = race_details[0].split('-')[0].replace('Race','').strip()
    race_date = datetime.datetime.strptime(''.join(race_details[1].split(',')[1:3]).strip(), '%B %d %Y')
    race_surface = race_details[2].split(',')[0].strip().lower()
    race_location = race_details[1].split(',')[3].strip().lower()   
    race_class = race_details[3].split(':')[-1].split('-')[-1].split(',')[-1].replace('Class','').strip().lower()

    out_path = os.path.join('data','6_prediction',f'{race_date.strftime("%Y_%m_%d")}_{race_no}.csv')
    if os.path.exists(out_path):
        continue

    if race_surface == 'all weather track':
        race_course = 'all weather track'
        race_distance = race_details[2].split(',')[1].replace('M','').strip().lower()
        try:
            race_going = race_details[2].split(',')[2].strip().lower()
        except:
            race_going = 'good'
    else:
        race_course = race_details[2].split(',')[1].replace('"','').replace('Course','').strip().lower()
        race_distance = race_details[2].split(',')[2].replace('M','').strip().lower()
        try:
            race_going = race_details[2].split(',')[3].strip().lower()
        except:
            race_going = 'good'

    num_horses = len(driver.find_elements(By.XPATH, '//*[@id="racecardlist"]/tbody/tr/td/table/tbody/tr'))

    for horse_idx, horse_row in enumerate(driver.find_elements(By.XPATH, '//*[@id="racecardlist"]/tbody/tr/td/table/tbody/tr')):
        horse_row = driver.find_elements(By.XPATH, f'//*[@id="racecardlist"]/tbody/tr/td/table/tbody/tr[{horse_idx + 1}]/td')
        
        jockey_name = horse_row[6].text.lower().split('(')[0].strip()
        if jockey_name == '-':
            continue
        
        race_dict['race_location'].append(race_location)
        race_dict['race_class'].append(race_class)
        race_dict['race_going'].append(race_going)
        race_dict['race_distance'].append(race_distance)
        race_dict['race_surface'].append(race_surface)
        race_dict['race_course'].append(race_course)

        race_dict['horse_handicap'].append(horse_row[5].text)
        race_dict['draw'].append(horse_row[8].text)
        race_dict['horse_rating'].append(horse_row[11].text)
        race_dict['horse_weight'].append(horse_row[13].text)
        race_dict['horse_age'].append(horse_row[16].text)
        race_dict['horse_gear'].append(int(horse_row[22].text.strip() != ''))
        race_dict['horse_import_type'].append(horse_row[-1].text.lower())

        # get jockey data
        jockey_file_path = os.path.join('data','1_scrape','jockeys',f'{jockey_name}.csv')
        if not os.path.exists(jockey_file_path):
            print(jockey_file_path, 'not exists, scraping...')

            # click on jockey window
            original_window = driver.current_window_handle
            driver.switch_to.window(original_window)
            while len(driver.window_handles) <= 2:
                horse_row[6].find_element(By.TAG_NAME, 'a').click()
            for window_handle in driver.window_handles:
                if window_handle != original_window:
                    driver.switch_to.window(window_handle)
                    break
            WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[2]/div[2]/table/tbody/tr').is_displayed())
            
            scrape_jockey(driver, jockey_name)

            # back to race card window
            for window_handle in driver.window_handles:
                if window_handle != original_window:
                    driver.switch_to.window(window_handle)
                    driver.close()
            driver.switch_to.window(original_window)

            # race_dict['jockey_age'].append(np.nan)
            # race_dict['jockey_nationality'].append(np.nan)
            # race_dict['jockey_wins'].append(np.nan)
            # race_dict['jockey_rides'].append(np.nan)
            # race_dict['jockey_stakes'].append(np.nan)
            # race_dict['jockey_same_race_wins'].append(np.nan)

        # else:
        jockey_df = pd.read_csv(jockey_file_path, index_col=0)
        try:
            race_dict['jockey_age'].append(int(jockey_df['jockey_age'].unique()[0]))
        except:
            race_dict['jockey_age'].append(np.nan)
        race_dict['jockey_nationality'].append(jockey_df['nationality'].unique()[0])
        race_dict['jockey_wins'].append(jockey_df['no. of wins'].unique()[0])
        race_dict['jockey_rides'].append(jockey_df['total rides'].unique()[0])
        race_dict['jockey_stakes'].append(str(jockey_df['stakes won'].unique()[0]).replace('$','').replace(',','').strip())

        race_distance = str(race_distance).replace('.0','')
        if 'sha tin' in race_location:
            try:
                race_dict['jockey_same_race_wins'].append(jockey_df[f'{race_surface}_{race_distance}_win'].unique()[0])
            except:
                race_dict['jockey_same_race_wins'].append(jockey_df[f'sha tin_all weather_{race_distance}_win'].unique()[0])
        else:
            race_dict['jockey_same_race_wins'].append(jockey_df[f'{race_location}_{race_distance}_win'].unique()[0])

        # click on horse window
        original_window = driver.current_window_handle
        driver.switch_to.window(original_window)
        while len(driver.window_handles) <= 2:
            horse_row[3].find_element(By.TAG_NAME, 'a').click()
            # href = horse_row.find_element(By.TAG_NAME, 'a').get_attribute('href')
            # driver.execute_script(href)
        for window_handle in driver.window_handles:
            if window_handle != original_window:
                driver.switch_to.window(window_handle)
                break
        WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[1]/table[1]/tbody/tr/td[2]/table/tbody/tr[6]/td[3]').is_displayed())

        race_dict['horse_origin'].append(driver.find_element(By.XPATH, f'//*[@id="innerContent"]/div[2]/div[1]/table[1]/tbody/tr/td[2]/table/tbody/tr[1]/td[3]').text.split(" / ")[0].lower())
        race_dict['horse_colour'].append(driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[1]/table[1]/tbody/tr/td[2]/table/tbody/tr[2]/td[3]').text.split(" / ")[0].lower())
        race_dict['horse_sex'].append(driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[1]/table[1]/tbody/tr/td[2]/table/tbody/tr[2]/td[3]').text.split(" / ")[1].lower())
        race_dict['total_stakes'].append(driver.find_element(By.XPATH, '//*[@id="innerContent"]/div[2]/div[1]/table[1]/tbody/tr/td[2]/table/tbody/tr[5]/td[3]').text.replace('$','').replace(',',''))

        last_row = len(driver.find_elements(By.XPATH, '//*[@id="innerContent"]/div[2]/div[1]/table[1]/tbody/tr/td[2]/table/tbody/tr'))
        horse_import_date = driver.find_element(By.XPATH, f'//*[@id="innerContent"]/div[2]/div[1]/table[1]/tbody/tr/td[2]/table/tbody/tr[{last_row}]/td[3]').text
        try:
            horse_import_date = datetime.datetime.strptime(horse_import_date, '%d/%m/%Y')
            race_dict['days_since_import'].append((race_date - horse_import_date).days)
        except:
            race_dict['days_since_import'].append(horse_import_date)

        # back to race card window
        for window_handle in driver.window_handles:
            if window_handle != original_window:
                driver.switch_to.window(window_handle)
                driver.close()
        driver.switch_to.window(original_window)

    # go to bet page and get odds
    while True:
        try:
            driver.get('https://bet.hkjc.com/racing/pages/odds_wp.aspx?lang=en')
            WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.ID, 'raceSel1').is_displayed())

            if driver.find_elements(By.XPATH, '//*[@id="container"]/div/div/div[2]/div[2]/div[1]/select/option'):
                for option in driver.find_elements(By.XPATH, '//*[@id="container"]/div/div/div[2]/div[2]/div[1]/select/option'):
                    if race_location in option.text.lower():
                        option.click()
                        WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.ID, 'winCell1').is_displayed())
                        break

            driver.find_element(By.ID, f'raceSel{race_no}').click()
            WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.ID, 'winCell1').is_displayed())
            time.sleep(1)
            break
        except:
            continue

    for horse_idx in range(num_horses):
        odds = driver.find_element(By.ID, f'winCell{horse_idx + 1}').text
        # odds = 'odds'
        try:
            race_dict['horse_odds'].append(float(odds))
            # race_dict['horse_odds'].append(odds)
        except:
            continue
        
    race_df = pd.DataFrame(race_dict)
    race_df.to_csv(out_path)
    print(out_path, 'scraped')

    driver.get("https://racing.hkjc.com/racing/information/English/Racing/RaceCard.aspx")
    WebDriverWait(driver, 10, 1).until(lambda x: x.find_element(By.ID, 'hplnkColSelect').is_displayed())

driver.close()

data/6_prediction/2024_01_28_1.csv scraped
data/6_prediction/2024_01_28_2.csv scraped
data/6_prediction/2024_01_28_3.csv scraped
data/6_prediction/2024_01_28_4.csv scraped
data/6_prediction/2024_01_28_5.csv scraped
data/6_prediction/2024_01_28_6.csv scraped
data/6_prediction/2024_01_28_7.csv scraped
data/6_prediction/2024_01_28_8.csv scraped
data/6_prediction/2024_01_28_9.csv scraped
data/6_prediction/2024_01_28_10.csv scraped
